In [15]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

In [16]:
#define current city
city = 'newberg'

In [18]:
#read in all labels geolocation data
geo = gpd.read_file('../../../data/{}/source/all.json'.format(city))

In [19]:
print(geo.total_bounds)

[-122.98861694   45.28316879 -122.92491913   45.33226395]


In [20]:
#read in cv metadata
meta= pd.read_json('../../../data/{}/source/cvMetadata.json'.format(city))

In [21]:
meta

,label_id,gsv_panorama_id,label_type_id,agree_count,disagree_count,notsure_count,pano_width,pano_height,pano_x,pano_y,canvas_width,canvas_height,canvas_x,canvas_y,zoom,heading,pitch,camera_heading,camera_pitch
0,18358,ddUJSyY5CduTNlEaq2Ktgg,1,0,0,0,13312.0,6656.0,3145,4320,720,480,376,234,1,213.250000,-27.812500,310.851898,0.472244
1,18402,6toXQIjLBocg5STjwGT7WA,7,0,0,0,16384.0,8192.0,8929,4879,720,480,473,291,1,356.000000,-10.000000,357.828888,-0.397575
2,410,jQKS0vBbswtm1o-31hP1MQ,7,0,0,0,13312.0,6656.0,6125,3558,720,480,138,228,2,1.526786,-7.455357,358.695679,-0.364601
3,386,HDBYsNqdb0gk1-iBjATMZg,7,0,0,0,13312.0,6656.0,6070,3616,720,480,267,218,2,319.294647,-9.598214,327.732300,3.463982
4,405,n-3v5kCz1Y0T5PgMnA2BgQ,7,0,0,0,13312.0,6656.0,7087,3634,720,480,483,252,2,1.526786,-7.455357,359.641052,-1.276314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16844,18337,ubiK-w3qZFH4YO4bI3415w,9,0,1,0,13312.0,6656.0,1959,4895,720,480,309,290,1,132.812500,-35.000000,249.037506,0.641014
16845,18338,ubiK-w3qZFH4YO4bI3415w,9,0,0,0,13312.0,6656.0,2627,3841,720,480,408,101,1,132.812500,-35.000000,249.037506,0.641014
16846,18339,ubiK-w3qZFH4YO4bI3415w,2,1,0,0,13312.0,6656.0,2008,3672,720,480,294,67,1,132.812500,-35.000000,249.037506,0.641014
16847,18340,ubiK-w3qZFH4YO4bI3415w,2,2,0,0,13312.0,6656.0,1541,3643,720,480,199,65,1,132.812500,-35.000000,249.037506,0.641014


In [23]:
meta = meta.drop(columns=['canvas_width','canvas_height','canvas_x','canvas_y','label_type_id','agree_count','disagree_count','notsure_count'])

In [24]:
#merge metadata and geo data on label_id
df = pd.merge(geo, meta, on='label_id')

In [27]:
#read in label-and-user-ids.csv
user_ids = pd.read_csv('../../../data/shared/label-and-user-ids.csv')
# select only city == current city
user_ids = user_ids[user_ids['city'] == city]

In [28]:
#megre user_ids and df on label_id
df = pd.merge(df, user_ids, on='label_id', how='inner')

In [29]:
#drop city column
df = df.drop(columns=['city'])

In [30]:
# break geometry into lat and lng
df['lat'] = df['geometry'].apply(lambda x: x.y)
df['lng'] = df['geometry'].apply(lambda x: x.x)

In [32]:
df

,label_id,gsv_panorama_id_x,label_type,severity,correct,has_validations,expired,high_quality_user,geometry,gsv_panorama_id_y,pano_width,pano_height,pano_x,pano_y,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng
0,145,0R2JTLibSCNsNb7aGNUzWg,NoSidewalk,NaN,None,False,True,True,POINT (-122.95290 45.31765),0R2JTLibSCNsNb7aGNUzWg,13312.0,6656.0,9123,3863,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904
1,18134,Z6Ja9-4wJxpQZPEnn8KSXA,NoSidewalk,4.0,None,False,False,True,POINT (-122.96400 45.31382),Z6Ja9-4wJxpQZPEnn8KSXA,13312.0,6656.0,6644,3744,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005
2,151,oqg9ULHZul23xfU6DwTRNg,NoSidewalk,NaN,None,False,False,True,POINT (-122.95360 45.31762),oqg9ULHZul23xfU6DwTRNg,13312.0,6656.0,7788,3776,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598
3,153,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,None,False,True,True,POINT (-122.95403 45.31742),WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,4663,3709,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025
4,154,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,None,False,True,True,POINT (-122.95419 45.31760),WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,7268,3600,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,5KKnsHQgODUBbIRZYiVbqg,NoSidewalk,NaN,None,False,False,True,POINT (-122.92580 45.30333),5KKnsHQgODUBbIRZYiVbqg,13312.0,6656.0,256,3757,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797
16843,8272,mdbgEoi-XTfZGiKqcWMlqw,NoSidewalk,NaN,None,False,True,True,POINT (-122.92559 45.29986),mdbgEoi-XTfZGiKqcWMlqw,3328.0,1664.0,2697,986,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591
16844,8281,BtyhcErvfI4z31FisBjeQg,NoSidewalk,NaN,None,False,False,True,POINT (-122.92661 45.30544),BtyhcErvfI4z31FisBjeQg,13312.0,6656.0,7561,3633,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605
16845,16595,--gG93FA53Lkv0H31-9Ruw,CurbRamp,NaN,True,True,False,True,POINT (-122.95637 45.29938),--gG93FA53Lkv0H31-9Ruw,13312.0,6656.0,1748,3983,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367


In [37]:
# save to csv
df.to_csv('../../../data/{}/processed-labels/labels_raw_{}.csv'.format(city,city), index = False)

In [38]:
#save df to shapefile
df.to_file('../../../data/{}/processed-labels/labels_raw_{}/labels_raw_{}.shp'.format(city,city,city))